# find (+follow) face

In [1]:
import sys
# customize path according to where sdk repo is in your system
sdk_path = '/home/david/GIT/cozmo-python-sdk/examples/tutorials/03_vision/'
sys.path.append(sdk_path)
from face_follow import follow_faces

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


In [2]:
sys.path.append('/home/david/GIT/cozmo-python-sdk/')
sys.path.append('/home/david/GIT/cozmo-python-sdk/src')
sys.path.append('/home/david/GIT/cozmo-python-sdk/src/cozmo')
from src import cozmo

In [3]:
cozmo.run_program(follow_faces, use_viewer=True, force_viewer_on_top=True)

2019-10-16 16:01:25,857 cozmo.general INFO     App connection established. sdk_version=1.4.11.dev0 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2019-10-16 16:01:25,859 cozmo.general INFO     Found robot id=1
2019-10-16 16:01:25,862 cozmo.general INFO     Connected to Android device serial=03160355293b2004
2019-10-16 16:01:26,056 cozmo.general INFO     Robot id=1 serial=024086a7 initialized OK


Found face!


2019-10-16 16:01:42,026 cozmo.general INFO     Shutting down connection
2019-10-16 16:01:42,029 cozmo.general INFO     Android serial=03160355293b2004 disconnected.


# find object

### select biggest bounding box

### re-orient